In [ ]:
!pip install FPDF
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from fpdf import FPDF
from google.colab import files

# Parâmetros
lambda_loss = 2.00
beta = 0.88
x_bruta = np.linspace(-500, 500, 1000, dtype=np.complex128)
n_values = [1, 2, 5, 10, 20, 50]

# Função
def value_function(x, beta, lambda_loss):
    return np.where(x >= 0, np.power(x, beta), -lambda_loss * np.power(-x, beta))

# Dataframe
data = {}
for n in n_values:
    data[f'n={n}'] = n**(1 - beta) * value_function(x_bruta, beta, lambda_loss).real

df = pd.DataFrame(data, index=x_bruta.real)
df.index.name = "x (Bruto)"

# Estatísticas gerais
df_stats = pd.DataFrame({
    "Média": df.mean(),
    "Mediana": df.median(),
    "Desvio Padrão": df.std(),
    "Mínimo": df.min(),
    "Máximo": df.max(),
    "CV (%)": (df.std() / df.mean()) * 100
})
df_stats.index.name = "n"

# Gráfico
plt.figure(figsize=(10, 6))
for n in n_values:
    plt.plot(x_bruta.real, df[f'n={n}'], label=f'n = {n}')
plt.axhline(0, color='black', linewidth=0.8, linestyle='--')
plt.axvline(0, color='black', linewidth=0.8, linestyle='--')
plt.xlabel("x")
plt.ylabel("Valor Percebido")
plt.title("Impacto da Diversificação na Percepção de Ganhos e Perdas")
plt.legend()
plt.grid()
plt.savefig("/content/grafico.png")
plt.close()

# Baixar gráfico
files.download("/content/grafico.png")

# Criar PDF
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)

pdf.add_page()
pdf.set_font("Arial", style='B', size=14)
pdf.cell(200, 10, "Análise da Diversificação na Percepção de Ganhos e Perdas", ln=True, align='C')

pdf.set_font("Arial", style='B', size=12)
pdf.cell(200, 10, "Gráfico de Diversificação", ln=True, align='C')

pdf.image("/content/grafico.png", x=10, y=40, w=180)

# Página para estatísticas
pdf.add_page()
pdf.set_font("Arial", style='B', size=12)
pdf.cell(200, 10, "Estatísticas Gerais", ln=True, align='C')

pdf.set_font("Arial", size=9)
columns = ["n", "Média", "Mediana", "Desvio Padrão", "Mínimo", "Máximo", "CV (%)"]
col_widths = [20, 30, 30, 30, 25, 25, 25]

pdf.ln(10)
pdf.set_x((210 - sum(col_widths)) / 2)
for col, width in zip(columns, col_widths):
    pdf.cell(width, 6, col, border=1, align='C')
pdf.ln()

for index, row in df_stats.iterrows():
    pdf.set_x((210 - sum(col_widths)) / 2)
    pdf.cell(20, 6, str(index), border=1, align='C')
    for i in range(6):
        pdf.cell(col_widths[i+1], 6, f"{row.iloc[i]:.2f}", border=1, align='C')
    pdf.ln()

# Página para tabela de resultados
pdf.add_page()
pdf.set_font("Arial", style='B', size=12)
pdf.cell(200, 10, "Tabela de Resultados", ln=True, align='C')

pdf.set_font("Arial", size=8)
pdf.ln(5)

# Cabeçalho das colunas
col_widths = [30] + [30] * len(n_values)
pdf.set_x((210 - sum(col_widths)) / 2)

pdf.cell(col_widths[0], 6, "x (Bruto)", border=1, align='C')
for n in n_values:
    pdf.cell(30, 6, f"n={n}", border=1, align='C')
pdf.ln()

# Exibir todos os resultados no PDF
for index, row in df.iterrows():
    pdf.set_x((210 - sum(col_widths)) / 2)
    pdf.cell(30, 6, f"{index:.2f}", border=1, align='C')
    for n in n_values:
        pdf.cell(30, 6, f"{row[f'n={n}']:.2f}", border=1, align='C')
    pdf.ln()

# Salvar PDF
pdf_path = "/content/analise_diversificacao.pdf"
pdf.output(pdf_path)

# Baixar PDF automaticamente
files.download(pdf_path)

print(f"\n📥 Download do PDF: {pdf_path}")
